In [0]:
%pip install langchain==0.1.17 langchain-openai==0.1.6 langchainhub==0.1.15 langgraph==0.0.40 databricks-vectorsearch==0.33 mlflow==2.12.1 lark==1.1.9 duckduckgo-search==5.3.0
dbutils.library.restartPython()

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
  Obtaining dependency information for duckduckgo-search==5.3.0 from https://files.pythonhosted.org/packages/01/0c/0d5bc90708f19dcc344dfa5afb3a2b612845fd9ce5d68f7a2a18a21ac46e/duckduckgo_search-5.3.0-py3-none-any.whl.metadata
  Obtaining dependency information for curl-cffi>=0.6.2 from https://files.pythonhosted.org/packages/a2/94/26f49de4ddc1085c71acefa9fbc8ecabc5b949055ab53217b8400a95012c/curl_cffi-0.6.3-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Obtaining dependency information for certifi>=2024.2.2 from https://files.pythonhosted.org/packages/ba/06/a07f096c664aeb9f01624f858c3add0a4e913d6c96257acb4fce61e7de14/certifi-2024.2.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/5.5 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━ 3.2/5.5 MB 95.8 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 5.4/5.5 MB 110.1 M

In [0]:
import os
from databricks.vector_search.client import VectorSearchClient
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import FunctionMessage, HumanMessage
from langchain_core.utils.function_calling import convert_to_openai_function, convert_to_openai_tool
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain.output_parsers.openai_tools import PydanticToolsParser
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain_openai import AzureChatOpenAI
from langchain import hub
from langchain_community.vectorstores import DatabricksVectorSearch
from langchain_community.embeddings import DatabricksEmbeddings
from langchain_community.chat_models import ChatDatabricks
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun
from operator import itemgetter

In [0]:
from langgraph.prebuilt import ToolExecutor, ToolInvocation
from langgraph.graph import StateGraph, END
from langchain_core.messages import BaseMessage, FunctionMessage
from langchain.prompts import PromptTemplate
import json
import operator
from typing import TypedDict, Annotated, Sequence

In [0]:
vs_endpoint_name = "edgar_vs_endpoint"
vs_index_fullname = "llm_hackathon.default.edgar_form_vs_index"
vsc = VectorSearchClient()

embedding_model = DatabricksEmbeddings(endpoint="databricks-bge-large-en")

vs_index = vsc.get_index(
    endpoint_name=vs_endpoint_name,
    index_name=vs_index_fullname
)

vectorstore = DatabricksVectorSearch(
    vs_index, text_column="content", embedding=embedding_model,
    columns=['name', 'tickers', 'exchanges', 'form', 'filing_date', 'industry']
)

chat_model = ChatDatabricks(endpoint="databricks-dbrx-instruct", temperature=0)

[NOTICE] Using a notebook authentication token. Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().


embedding model is not used in delta-sync index with Databricks-managed embeddings.


In [0]:
metadata_field_info = [
    AttributeInfo(
        name="name",
        description="The name of the company",
        type="string",
    ),
    AttributeInfo(
        name="tickers",
        description="The ticker symbols of the company",
        type="string",
    ),
    AttributeInfo(
        name="exchanges",
        description="The stock exchange where the stock is traded",
        type="string",
    ),
    AttributeInfo(
        name="form",
        description="The filing forms (10-K or 10-Q)",
        type="string"
    ),
    AttributeInfo(
        name="filing_date",
        description="The filing date of the form",
        type="date"
    ),
    AttributeInfo(
        name="industry",
        description="The industry of the company",
        type="string"
    ),
]

document_content_description = "The sec filing of financial statements and report of the company."

retriever = SelfQueryRetriever.from_llm(
    chat_model, vectorstore, document_content_description, metadata_field_info, search_kwargs={"k" : 10},
)

In [0]:
RAG_TEMPLATE = """\
You are an assistant for financial analyst. You are answering finance question about company's news, stock and financial reports (10-K,  10-Q forms). If the question is not related to one of these topics, kindly decline to answer. If the context is empty or you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as concise as possible.

Use the following pieces of context to answer the question at the end:
{context}

Question: {question}
Answer:
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_TEMPLATE)

self_query_retrieval_chain = (
    {"context": itemgetter("question") | retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | chat_model, "context": itemgetter("context")}
)

In [0]:
os.environ["AZURE_OPENAI_ENDPOINT"] = dbutils.secrets.get(scope="development", key="azure_openai_endpoint")
os.environ["AZURE_OPENAI_API_KEY"] = dbutils.secrets.get(scope="development", key="azure_openai_api_key")

prompt = hub.pull("hwchase17/openai-functions-agent")

llm = AzureChatOpenAI(
    api_version=dbutils.secrets.get(scope="development", key="azure_openai_chat_deployment_version"),
    azure_deployment=dbutils.secrets.get(scope="development", key="azure_openai_chat_deployment_name"),
    temperature=0,
)

In [0]:
search_tool = DuckDuckGoSearchRun()
retriever_tool = create_retriever_tool(
    retriever,
    "sec_filing_search",
    "Search for sec filing forms or financial information about any company.",
)

tools = [retriever_tool, search_tool]

In [0]:
agent = create_tool_calling_agent(llm, tools, prompt)

In [0]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [0]:
agent_executor.invoke({"input": "what was alphabet's revenue?"})



> Entering new AgentExecutor chain...

Invoking: `sec_filing_search` with `{'query': 'Alphabet revenue'}`


| Revenue | | | $ | 1,194,773 |  |  | | | $ | 988,608 |  |  | | |  | | |  | | |  | | |
| Revenue as a dollar change | | |  | | |  | | | $ | 206,165 |  |  | | |  | | |  | | |  | | |
| Revenue as a percentage change | | |  | | |  | | | 21 | | % |  | | |  | | |  | | |  | | |  
Revenue for the three months ended March 31, 2024 increased $206.2 million compared to the same period in 2023. The increase was primarily due to a combination of growth in advertisers, optimization efficiencies, and improvement in auction-based advertising demand. The increase was also driven by higher subscription revenue due to growth in the number of subscribers. Cost of Revenue  
|  |  |  |  |  |  |  |  |  |  |  |  |  | | |  | | |  | | |  | | |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
|  | | | Three 

{'input': "what was alphabet's revenue?",
 'output': "Alphabet's revenue for the three months ended March 31, 2024, was approximately $1,194,773,000 (in thousands), which represents an increase of $206.2 million compared to the same period in 2023. This increase was primarily due to growth in advertisers, optimization efficiencies, improvement in auction-based advertising demand, and higher subscription revenue due to growth in the number of subscribers."}

In [0]:
agent_executor.invoke({"input": "what is alphabet's management outlook of the future?"})



> Entering new AgentExecutor chain...

Invoking: `sec_filing_search` with `{'query': 'Alphabet management outlook'}`


ITEM 5.OTHER INFORMATION 10b5-1 Trading PlansDuring the fiscal quarter ended March 31, 2024, no Section 16 director or officer adopted, modified, or terminated a “Rule 10b5-1 trading arrangement” (as defined in Item 408 of Regulation S-K of the Exchange Act).There were no “non-Rule 10b5-1 trading arrangements” (as defined in Item 408 of Regulation S-K of the Exchange Act) adopted, modified or terminated during the fiscal quarter ended March 31, 2024 by our directors and Section 16 officers. Compensatory Arrangements of Certain OfficersOn April 16, 2024, the Leadership Development, Inclusion and Compensation Committee of the Board of Directors of Alphabet approved the accrual of dividend equivalent units to holders of all unvested stock units, subject to the approval of a dividend declaration by the Board of Directors of the Company (which was approved and announced o

{'input': "what is alphabet's management outlook of the future?",
 'output': "Alphabet's management outlook on the future, as derived from their SEC filings, highlights several key areas of focus and concern:\n\n1. **Competitive Landscape**: Alphabet acknowledges the intense and growing competition in the market for its products and services. The company faces challenges from current competitors who address different aspects of its offerings, as well as from new entrants and existing large companies seeking to expand their offerings. Alphabet's main sources of competition include AI companies focused on conversational interfaces, language understanding, and processing; organizations offering products within Alphabet's target verticals; and legacy providers, including large technology providers seeking to add or scale AI capabilities.\n\n2. **Competitive Advantages and Challenges**: The company is aware that many of its competitors may possess advantages such as higher brand visibility,

In [0]:
agent_executor.invoke({"input": "what was apples's revenue?"})



> Entering new AgentExecutor chain...

Invoking: `sec_filing_search` with `{'query': 'Apple revenue'}`


| China |  |  | 98,000 |  |  |  | 1,984,000 |  |
| United Arab Emirates |  |  | 570,000 |  |  |  | – |  |
| Vietnam |  |  | 225,000 |  |  |  | 515,000 |  |
| Other |  |  | 2,523,000 |  |  |  | 1,343,000 |  |
| Total |  | $ | 9,445,000 |  |  | $ | 10,483,000 |  |  
The following table summarizes the revenues by product line and type:  
|  |  | Year Ended December 31, 2023 | | | | | | | | | | | | | |  |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
|  |  | Device Revenue | |  |  | Service Revenue | |  |  | Other Revenue | |  |  | Total Revenue | |  |
|  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |  |
| AXP |  | $ | 4,970,000 |  |  | $ | 232,000 |  |  | $ | – |  |  | $ | 5,202,000 |  |
| BioArchive |  |  | 1,484,000 |  |  |  | 1,361,000 |  |  |  | – |  |  |  | 2,845,000 |  |
| X-Series |  |  | 121,000 |  |  |  | 152,000 |  |

{'input': "what was apples's revenue?",
 'output': "Apple's total revenue for the year ended December 31, 2023, was $9,445,000 (in thousands), which represents a decrease from the total revenue of $10,483,000 (in thousands) for the year ended December 31, 2022."}

In [0]:
self_query_retrieval_chain.invoke({'question': "what was alphabet's revenue?"})

{'response': AIMessage(content="Based on the provided documents, Alphabet Inc.'s revenues for the three months ended March 31, 2024 were $80,539 million.", response_metadata={'prompt_tokens': 5586, 'completion_tokens': 30, 'total_tokens': 5616}, id='run-9558169b-49b9-48aa-a59f-6dd797a162e6-0'),
 'context': [Document(page_content='|  | | | Three Months Ended | | | | | | | | |\n|  | | | March 31, | | | | | | | | |\n|  | | | 2023 | | |  | | | 2024 | | |\n| Google Search & other | | | $ | 40,359 |  |  | | | $ | 46,156 |  |\n| YouTube ads | | | 6,693 | |  |  | | | 8,090 | |  |\n| Google Network | | | 7,496 | |  |  | | | 7,413 | |  |\n| Google advertising | | | 54,548 | |  |  | | | 61,659 | |  |\n| Google subscriptions, platforms, and devices | | | 7,413 | |  |  | | | 8,739 | |  |\n| Google Services total | | | 61,961 | |  |  | | | 70,398 | |  |\n| Google Cloud | | | 7,454 | |  |  | | | 9,574 | |  |\n| Other Bets | | | 288 | |  |  | | | 495 | |  |\n| Hedging gains (losses) | | | 84 | |  |  |

In [0]:
self_query_retrieval_chain.invoke({'question': "what was amazon's revenue?"})

{'response': AIMessage(content="I'm happy to help with questions about company news, stocks, and financial reports. However, I don't have real-time data access, so I can't provide the current revenue of Amazon. You can find this information in Amazon's latest financial report (10-K or 10-Q form).", response_metadata={'prompt_tokens': 337, 'completion_tokens': 59, 'total_tokens': 396}, id='run-0c98e517-a9a2-47c4-b3ce-881bdebd4b8b-0'),
 'context': []}

In [0]:
search_tool.run("what was amazon's revenue?")

"Amazon.com, Inc. (NASDAQ: AMZN) today announced financial results for its fourth quarter ended December 31, 2023. Fourth Quarter 2023 Net sales increased 14% to $170.0 billion in the fourth quarter, compared with $149.2 billion in fourth quarter 2022. Excluding the $1.3 billion favorable impact from year-over-year changes in foreign exchange rates throughout the quarter, net sales increased 13 ... The company had $143.3 billion in revenue in the first three months of the year, up 13 percent from a year earlier. Profit more than tripled, to $10.4 billion. The results beat analysts ... From 2004 to 2023, the net revenue of Amazon e-commerce and service sales has increased tremendously. In the last reported year, the multinational e-commerce company's net revenue was almost 576 ... The steadily increasing sales figures speak for themselves — Amazon hit nearly 575 billion U.S. dollars in net sales revenue worldwide in 2023. Due to its global scope and reach, Amazon is ... Amazon Revenue. 

In [0]:
tools = [search_tool]
tool_executor = ToolExecutor(tools)
functions = [convert_to_openai_function(t) for t in tools]

model = AzureChatOpenAI(
    api_version=dbutils.secrets.get(scope="development", key="azure_openai_chat_deployment_version"),
    azure_deployment=dbutils.secrets.get(scope="development", key="azure_openai_chat_deployment_name"),
    temperature=0,
    streaming=True
)
model = model.bind_functions(functions)

In [0]:
class AgentState(TypedDict):
  messages: Annotated[Sequence[BaseMessage], operator.add]

def call_model(state):
  messages = state["messages"]
  response = model.invoke(messages)
  return {"messages" : [response]}

def call_tool(state):
  last_message = state["messages"][-1]

  action = ToolInvocation(
      tool=last_message.additional_kwargs["function_call"]["name"],
      tool_input=json.loads(
          last_message.additional_kwargs["function_call"]["arguments"]
      )
  )

  response = tool_executor.invoke(action)

  function_message = FunctionMessage(content=str(response), name=action.tool)

  return {"messages" : [function_message]}

def should_continue(state):
  last_message = state["messages"][-1]

  if "function_call" not in last_message.additional_kwargs:
    return "end"

  return "continue"

In [0]:
workflow = StateGraph(AgentState)

workflow.add_node("agent", call_model)
workflow.add_node("action", call_tool)
workflow.set_entry_point("agent")

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue" : "action",
        "end" : END
    }
)

workflow.add_edge("action", "agent")

In [0]:
app = workflow.compile()

In [0]:
question = "what was apples's revenue in the most recent quarter?"

inputs = {"messages" : [HumanMessage(content=question)]}

app.invoke(inputs)

{'messages': [HumanMessage(content="what was apples's revenue in the most recent quarter?"),
  AIMessage(content="As of my last update in April 2023, I can't provide real-time data or updates. To find Apple's most recent quarterly revenue, I recommend checking the latest financial reports on Apple's investor relations page or using financial news websites like Bloomberg, Reuters, or CNBC. These sources frequently update with the latest earnings reports from major companies like Apple.", response_metadata={'finish_reason': 'stop'}, id='run-b7b057e1-000a-472f-96e7-d4c240479465-0')]}

In [0]:
def convert_state_to_query(state_object):
  return {"question" : state_object["messages"][-1].content}

def convert_response_to_state(response):
  return {"messages" : [response["response"]]}

langgraph_node_rag_chain = convert_state_to_query | self_query_retrieval_chain | convert_response_to_state

In [0]:
rag_agent = StateGraph(AgentState)

rag_agent.add_node("agent", call_model)
rag_agent.add_node("action", call_tool)
rag_agent.add_node("first_action", langgraph_node_rag_chain)
rag_agent.set_entry_point("first_action")

In [0]:
def is_fully_answered(state):

    ### Extract the question and response from our RAG pipeline
    question = state["messages"][0].content
    answer = state["messages"][-1].content

    ### Create a Pydantic model to capture our LLMs response
    class answered(BaseModel):
        binary_score: str = Field(description="Fully answered: 'yes' or 'no'")

    ### Create and bind our tool to our model
    answered_tool = convert_to_openai_tool(answered)
    model = AzureChatOpenAI(
        api_version=dbutils.secrets.get(scope="development", key="azure_openai_chat_deployment_version"),
        azure_deployment=dbutils.secrets.get(scope="development", key="azure_openai_chat_deployment_name"),
        temperature=0,
    )
    model = model.bind(
        tools=[answered_tool],
        tool_choice={"type" : "function", "function" : {"name" : "answered"}}
    )

    ### We'll want to parse the output into a usable format
    parser_tool = PydanticToolsParser(tools=[answered])

    prompt = PromptTemplate(
        template="""You will determine if the question is fully answered by the response.\n
        Question:
        {question}

        Response:
        {answer}

        You will respond with either 'yes' or 'no'.""",
        input_variables=["question", "answer"])

    ### Classic LCEL chain!
    fully_answered_chain = prompt | model | parser_tool

    response = fully_answered_chain.invoke({"question" : question, "answer" : answer})

    if response[0].binary_score == "no":
        return "continue"

    return "end"

In [0]:
rag_agent.add_conditional_edges(
    "first_action",
    is_fully_answered,
    {
        "continue" : "agent",
        "end" : END
    }
)

rag_agent.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue" : "action",
        "end" : END
    }
)

rag_agent.add_edge("action", "agent")
rag_agent_app = rag_agent.compile()

In [0]:
question = "what was amazon's revenue?"

inputs = {"messages" : [HumanMessage(content=question)]}

rag_agent_app.invoke(inputs)

{'messages': [HumanMessage(content="what was amazon's revenue?"),
  AIMessage(content="I'm happy to help with questions about company news, stocks, and financial reports. However, I don't have real-time data access, so I can't provide the current revenue of Amazon. You can find this information in Amazon's latest financial report (10-K or 10-Q form).", response_metadata={'prompt_tokens': 337, 'completion_tokens': 59, 'total_tokens': 396}, id='run-2b4b44a0-8e4d-4c96-b738-721f5b99518f-0'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"Amazon revenue 2022"}', 'name': 'duckduckgo_search'}}, response_metadata={'finish_reason': 'function_call'}, id='run-a2904fb1-8820-4313-838c-8fec77068cdb-0'),
  FunctionMessage(content="In 2022, Amazon's annual revenue peaked with nearly 514 billion U.S. dollars, up from 470 billion U.S. dollars in 2021. On February 1, 2024, Amazon.com Inc ( NASDAQ:AMZN) released its impressive fourth-quarter earnings, showcasing signi